In [1]:
import requests
import pickle
import copy
import datetime

ModuleNotFoundError: No module named 'requests'

In [ ]:
verbose = True

In [ ]:
# This class maps a dictionary into a simple Python object
class Dict2Obj(object):
    '''
    Class to transform a dict into an obj.
    
    Dict keys must be string which becomes the obj attributes' names.
    '''
    
    def __init__(self, dic):
        for key in dic:
            setattr(self, key, dic[key])
        else:
            return None
    def __repr__(self):
        return str(self.__dict__)
    def __copy__(self):
        return Dict2Obj(self.__dict__)

In [ ]:
# Please launch the server before continuing
url_exploration = 'http://localhost:5044/api/exploration'
url_function_request = 'http://localhost:5044/api/function_request'

In [ ]:
def generic_function(submodule_name, file_name, function_name, method_name = None, **args):
    '''
    Post a request to the server asking to execute the corresponding remote function.
    
    :param str submodule_name: the name of the submodule, which contains the function to be executed

    :param str file_name: the name of the file, which contains the function to be executed

    :param str function_name: the name of the function to be executed

    :param str method_name: if this parameter is not None, it means function_name is actually the name of a class ;
    thus method_name contains the name of the method of the class to be executed.

    :param dict **args: is a dict of arguments for the function we want to execute. Besides these arguments, it may also
    contains:
        - a variable _dic, which contains the object's __dict__, and which is passed through the API
        - a variable _obj, which contains the object itself. It cannot be passed through the API, but it is used to
        locally modify the object after the result has been returned by the API.

    :return: result from the remote function.

    :raise: Exception if the POST request returns an error.
    '''

    args = dict((k, v) for (k, v) in args.items() if v != '__$__')
    
    # for special methods (such as __repr__), the obj is passed in _dic, so we need to pop it out and replace it
    # by its __dict__
    if method_name is not None and method_name[:2] == '__':
        obj = args['_dic']
        args['_dic'] = dict((k, v) for (k, v) in args['_dic'].__dict__.items() if not hasattr(v, "__call__"))
    else:    
        obj = args.pop("_obj") if "_obj" in args else None

    dic = {'module':'main_module.' + submodule_name + '.' + file_name, 'function':function_name, 'args':args}

    if method_name:
        dic['method'] = method_name 
    req = requests.post(url_function_request, data=pickle.dumps(dic))
    ingoing = pickle.loads(req.content)

    if req.status_code == 400 or req.status_code == 521:
        print(req.json()['error'])
        raise Exception(req.json()['error'])
    
    if req.status_code == 201:
        if verbose:
            print('creating class dynamically...')
        instance = Dict2Obj(ingoing['_dic'])   # populate instance with its __dict__
        methods = ingoing['methods']
        for method_name, list_of_args in methods.items():   # populate instance with its methods
            if method_name != '__init__':    # we don't want to have access to the __init__ here.
                list_of_args.insert(0, "_dic")
                function = dic_to_func(submodule_name, file_name, function_name, list_of_args, method_name=method_name)
                setattr(instance, method_name, function)
        return change_instance_functions_to_pass_automatically_obj_and_dic(instance, function_name)
    
    elif req.status_code == 202:
        if verbose:
            print('updating object...')
        method_result = ingoing['method_result']
        obj.__dict__.update(ingoing['_dic'])    # updating the instance __dict__
        return method_result
    
    else:        
        return ingoing


In [ ]:
def dic_to_func(submodule_name, file_name, function_name, list_of_args, method_name=None):
    '''String manipulations to create lambda function from its signature.'''

    s = 'function = lambda '
    for attr in [x for x in list_of_args if x != '**kwargs']:
        s += attr + ','
    s += '**kwargs: generic_function("' + '", "'.join([submodule_name, file_name, function_name])
    if method_name:
        s += '", "' + method_name
    s += '", **{'
    for attr_name in [x.split('=')[0] for x in list_of_args if x != '**kwargs']:
        s += '"' + attr_name + '":' + attr_name + ','
    s = s[:-1] + "}, **kwargs)"
    print(s)
    exec(s)
    return locals()['function']

In [ ]:
def change_instance_functions_to_pass_automatically_obj_and_dic(instance, function_name):
    '''String manipulations to pass the _obj and the _dic arguments to the lambda function, \
    without the user having to worry about it.'''

    if verbose:
        print('in change_instance_functions_to_pass_automatically_obj_and_dic')
    
    global _dict_of_variables
    try:
        _dict_of_variables
    except:
        _dict_of_variables = {}
  #  if "_dict_of_variables" not in globals():
  #      globals()['_dict_of_variables'] = {}
    
    copy_of_instance = copy.copy(instance)
    
    tag = str(datetime.datetime.now())
    _dict_of_variables[tag + '-0'] = instance
    _dict_of_variables[tag + '-1'] = copy_of_instance
    counter = 1
    
    dic_special_method = {}
    
    for name, func in instance.__dict__.items():
        if hasattr(func, '__call__'):
            counter += 1
            _dict_of_variables[tag + '-' + str(counter)] = name
            
            s = 'instance.temp = lambda '
            for attr in [x for x in list(func.__code__.co_varnames) if x not in ['_dic', 'kwargs']]:
                s += attr + '="__$__",'
            s += '**kwargs: getattr(_dict_of_variables["' + tag + '-0"], _dict_of_variables["' + tag + '-' + str(counter) + '"])'
            s += '(_obj = _dict_of_variables["' + tag + '-1"], _dic=dict((k, v) for (k, v) in _dict_of_variables["' + tag + '-1"].__dict__.items() if not hasattr(v, "__call__")),**{'
            for attr_name in [x.split('=')[0] for x in list(func.__code__.co_varnames) if x not in ['_dic', 'kwargs']]:
                s += '"' + attr_name + '":' + attr_name + ','
            if s[-1] == ',':
                s = s[:-1]
            s += '}, **kwargs)'
            print(s)
            exec(s)
            if name[:2] != '__':
                setattr(copy_of_instance, name, instance.temp)
            else:
                # for special methods such as __repr__, we stack them in dic_special_method, which is passed
                # to the class and not to the instance
                dic_special_method[name] = func
                del copy_of_instance.__dict__[name]
            delattr(instance, 'temp')
            
        
            
    copy_of_instance.__class__ = type(function_name, (Dict2Obj,), dic_special_method)
    return copy_of_instance


In [ ]:
# here we request for the JSON describing the remote file structure.
# The whole code is based on a precise file structure (see remote module for an example).
# It could probably be generalized even further, but that's not worth the trouble for now.

r = requests.get(url_exploration)
print("status code: " + str(r.status_code))
print(r.json())

In [ ]:
def create_class_from_name(submodule_name):
    '''Instanciate and return an object that reproduces the structure of a remote submodule.'''

    r = requests.get(url_exploration)
    dic_files = r.json()[submodule_name]
    for file_name, dic_function_names in dic_files.items():
        for function_name, list_of_args in dic_function_names.items():
            if type(list_of_args) == dict:
                list_of_args = list_of_args["__init__"]
            dic_files[file_name][function_name] = dic_to_func(submodule_name, file_name, function_name, list_of_args)
            
    obj = Dict2Obj(dic_files)
    for k, v in obj.__dict__.items():
        setattr(obj, k, Dict2Obj(v))
    return obj

In [ ]:
# here we create local proxy for distant submodule1
submodule1 = create_class_from_name('submodule1')

In [ ]:
# we run one function of the submodule
submodule1.file1.hello(3, 4, s=5)

In [ ]:
# we instanciate a class of the submodule
x = submodule1.file1.Hallo(name="Pierre")

In [ ]:
# the type is the good one (ninja trick to do that...)
type(x)

In [ ]:
x.__dict__

In [ ]:
# the object has both attributes and methods (to be precise, it has no method, but rather functions)
x

In [ ]:
x.name = 'Paul'

In [ ]:
f = x.__repr__

In [ ]:
f()

In [ ]:
x.__dict__

In [ ]:
x

In [ ]:
x.sentence

In [ ]:
# below lies the magic ! We can call remote methods on objects and they will be executed almost as usual.
# The method can do two things (and in this example the method 'polite' actually does these two things) : 
# - return a result (which is in the return statement in the remote method)
# - modify in-place the object calling the method (here the 'sentence' attribute is modified)
# The only restriction is that the methods of the object cannot be modified in-place, for now. 
# But we could imagine a solution to this problem (which is quite far-fetched anyway) by using some kind of flag in the server.  

# Two delicate things are handled behind the scenes : the **kwargs are working ('s' is a kwarg here), and the by-default
# arguments are working as well (if you look at polite, you will see it has a by-default 't' argument)

res = x.polite(toto='Marion', s=2)

In [ ]:
res

In [ ]:
x

In [ ]:
res = x.polite(toto='42')
print(res)

In [ ]:
x

In [ ]:
# another example, without all the intermediate steps. 
# The only weird thing compared to what we could expect is the [1]. All the rest is normal.
# This line requires two requests : one to instantiate the object of the class Hallo(), and one to execute the method 'polite'
obj = submodule1.file1.Hallo(name="Olivier")
print(obj.polite('Borderies'))
obj.sentence

In [ ]:
x.test()

In [ ]:
_dict_of_variables